# Extracting triples
The model outputs relationship triples in the form of structured text. This notebook shows how the relationship triples are extracted out of that text.

In [1]:
from datasets import load_dataset
import evaluate
import yaml
import re
import pandas as pd
from transformers import AutoTokenizer
from wasabi import msg
from pathlib import Path
from os.path import abspath

/home/lgrootde/Generative-re-tests/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting home directory

In [2]:
home_dir = Path(abspath("")).parent

msg.info(f"Home directory: {home_dir}")

# Change when using a different config
config_path = home_dir.joinpath('config/config_testing.yaml')

ℹ Home directory: /home/lgrootde/Generative-re-tests


## Load Config

In [3]:
# Load the config
with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

## Load dataset

In [4]:
dataset = load_dataset(
        config['dataset_vars']['type'], 
        data_dir=home_dir.joinpath(config['dataset_vars']['dir']),
        column_names=config['dataset_vars']['column_names']
        )

dataset_train = dataset['train'].select(range(1,501)) # remove first row that contains column names
dataset_eval = dataset['validation'].select(range(1,501)) # remove first row that contains column names

In [5]:
relations_in_structerd_text = dataset_train["relations"]
entity_types = {"chemical":"@CHEMICAL@", 
                "disease":"@DISEASE@"}
rel_types = {"chemical induced disease":"@CID@"}

In [6]:
# voorbeeld
relations_in_structerd_text[4]

'lithium ; li @CHEMICAL@ chronic renal failure @DISEASE@ @CID@ lithium ; li @CHEMICAL@ proteinuria @DISEASE@ @CID@ lithium ; li @CHEMICAL@ hypertension @DISEASE@ @CID@'

## Extracting triples from structerd text

In [7]:
def split_on_labels(input_text, labels):
    '''
    Split structered text into the structerd text for the entities and the relation label
    
    input:
        input_text: str, structerd text representing a collection of relationships.
        labels: list, a list of the relation labels to split the text on.

    returns:
        a list of strings which alternates ner text and re label:
        ["ner_text", "re_label", "ner_text", ect...]
    '''
    # Escape labels to ensure special characters are treated as literals in regex
    escaped_labels = [re.escape(label) for label in labels]
    # Join the labels into a regex pattern with alternation to match any of them
    pattern = '|'.join(escaped_labels)
    # Use re.split() with the compiled pattern, keeping the delimiters in the result
    relation_segments = re.split(f'({pattern})', input_text)
    # Filter out empty strings that might result from splitting
    relation_segments = [segment for segment in relation_segments if segment]
    return relation_segments

In [8]:
# Testing split_on_labels
print(relations_in_structerd_text[10])

split_on_labels(relations_in_structerd_text[10], ['@CID@'])

chloroacetaldehyde ; caa @CHEMICAL@ hemorrhagic @DISEASE@ @CID@ chloroacetaldehyde ; caa @CHEMICAL@ cystitis @DISEASE@ @CID@


['chloroacetaldehyde ; caa @CHEMICAL@ hemorrhagic @DISEASE@ ',
 '@CID@',
 ' chloroacetaldehyde ; caa @CHEMICAL@ cystitis @DISEASE@ ',
 '@CID@']

In [9]:
def handle_coreforents(ent, keep):
    '''
    Will split entity text if it contains a coferent mention.
    
    input:
        ent: a tuple of ("entity_text", "label")
        keep: boolean, wheter to keep the coferent mentions or not. if not only the first coferent mentions will be kept.
    returns:
        an tuple with (("entity texts",) "label")
    '''
    coreferents = tuple([coref.strip() for coref in ent[0].split(';')])
    if keep and len(coreferents) > 1:
        return (coreferents, ent[1]) 
    else:
        return ((coreferents[0],), ent[1]) # Only keep the first coferent mention

In [10]:
# Testing handle_coreforents
ents = split_on_labels(relations_in_structerd_text[10], ['@CID@'])[::2]
print(f"split relations: {ents}\n")
ent = re.findall(r'(.+?)\s@(\w+)@', ents[0])
print(f"Named entities of a single relation: {ent}\n")


print(f"With a coferent mention: {handle_coreforents(ent[0], True)}")
print(f"Without a coferent mention:{handle_coreforents(ent[1], True)}")

split relations: ['chloroacetaldehyde ; caa @CHEMICAL@ hemorrhagic @DISEASE@ ', ' chloroacetaldehyde ; caa @CHEMICAL@ cystitis @DISEASE@ ']

Named entities of a single relation: [('chloroacetaldehyde ; caa', 'CHEMICAL'), (' hemorrhagic', 'DISEASE')]

With a coferent mention: (('chloroacetaldehyde', 'caa'), 'CHEMICAL')
Without a coferent mention:(('hemorrhagic',), 'DISEASE')


In [11]:
def extract_relation_triples(text: str, re_labels: list[str], keep_coreforents: bool = False) -> list[dict]:
    '''
    This function extracts the relationship triples out of structerd text. 
    This function assumes that the NER labels are in this structure: @label@
    
    input:
    text: The structerd text as a string.
    re_labels: The relationship labels.

    returns:
    A list of dictionaries
    '''
    # Split the input text into relation segments
    relation_segments = split_on_labels(text, re_labels)
    
    # Remove the last empty segment if it exists
    if not relation_segments[-1].strip():
        relation_segments = relation_segments[:-1]

    # Map relation label to entity text
    entity_texts = relation_segments[::2] # All uneven elements
    relation_labels = relation_segments[1::2] # All even elements
    if len(entity_texts) != len(relation_labels):
        msg.warn(f" len entity text {len(entity_texts)} | {entity_texts}")
        msg.warn(f" len relation labels {len(relation_labels)} | {relation_labels}")
        raise ValueError('Amount of relation labels in the text does not equal to amount of entities pairs')
        
    
    # Initialize a list to hold the relation triples
    relations = []
    
    for entity_text, re_label in zip(entity_texts, relation_labels):
        # Split head and tail entities and their labels
        head_ent, tail_ent = [handle_coreforents(ent, keep_coreforents) for ent in re.findall(r'(.+?)\s@(\w+)@', entity_text)]
        # print(f"head_ent {head_ent} | tail_ent {tail_ent}") #DEBUG
        
        re_label = re_label.split('@')[1]
        relations.append({
            're_label':re_label,
            'head_ent': {'label':head_ent[1], 'text':head_ent[0]},
            'tail_ent': {'label':tail_ent[1], 'text':tail_ent[0]}
        })
    
    return relations

In [12]:
# Example usage
input_text = 'lithium ; li @CHEMICAL@ chronic renal failure @DISEASE@ @CID@ lithium ; li @CHEMICAL@ proteinuria @DISEASE@ @CID@ lithium ; li @CHEMICAL@ hypertension @DISEASE@ @CID@'
relation_triples = extract_relation_triples(input_text, ['@CID@'], True)
relation_triples

[{'re_label': 'CID',
  'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')},
  'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}},
 {'re_label': 'CID',
  'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')},
  'tail_ent': {'label': 'DISEASE', 'text': ('proteinuria',)}},
 {'re_label': 'CID',
  'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')},
  'tail_ent': {'label': 'DISEASE', 'text': ('hypertension',)}}]

In [13]:
%%time
relation_triples = [extract_relation_triples(i, ['@CID@'], True) for i in relations_in_structerd_text]

CPU times: user 4.6 ms, sys: 697 µs, total: 5.3 ms
Wall time: 5.24 ms


In [14]:
dataframe=pd.DataFrame({'structerd_text': relations_in_structerd_text, 'Relation triples': relation_triples})
dataframe

,structerd_text,Relation triples
0,alpha-methyldopa @CHEMICAL@ hypotensive @DISEA...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
1,lidocaine @CHEMICAL@ cardiac asystole @DISEASE...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
2,suxamethonium ; suxamethonium chloride ; sch @...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
3,scopolamine ; hyoscine @CHEMICAL@ overdosage @...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
4,lithium ; li @CHEMICAL@ chronic renal failure ...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
...,...,...
495,zonisamide @CHEMICAL@ visual hallucinations @D...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
496,pan ; puromycin aminonucleoside @CHEMICAL@ nep...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
497,ticlopidine @CHEMICAL@ aplastic anemia @DISEAS...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."
498,scopolamine @CHEMICAL@ amnesia @DISEASE@ @CID@...,"[{'re_label': 'CID', 'head_ent': {'label': 'CH..."


## Dealing with unstructured text

### Load dataset of predicted output

In [15]:
test_eval_dataset = load_dataset(
        config['dataset_vars']['type'], 
        data_dir=home_dir.joinpath("data/testing_eval/cdr_seq2rel"),
        data_files={'test_trained': 'test_trained.csv', 
                    'test_untrained':'test_untrained.csv', 
                    'test_trained_gc1': 'test_trained_gc1.csv',
                    'test_re100':'test_re100.csv',
                    'test_re53_ner53':'test_re53_ner53.csv',
                    'test_trained_t5_3b':"test_trained_T5_3b.csv"},
        column_names=["input", "expected output", "predicted"]
        )

test_eval_dataset_trained = test_eval_dataset['test_trained'].select(range(1,501)) # remove first row that contains column names
test_eval_dataset_trained_gc1 = test_eval_dataset['test_trained'].select(range(1,501)) # remove first row that contains column names
test_eval_dataset_untrained = test_eval_dataset['test_untrained'].select(range(1,501)) # remove first row that contains column names
test_eval_dataset_re100 = test_eval_dataset['test_re100'].select(range(1,11)) # remove first row that contains column names
test_eval_dataset_re53_ner53 = test_eval_dataset['test_re53_ner53'].select(range(1,11)) # remove first row that contains column names
test_eval_dataset_trained_3b = test_eval_dataset['test_trained_t5_3b'].select(range(1,501)) # remove first row that contains column names

In [16]:
for row in test_eval_dataset_re53_ner53:
    msg.info(f"Artificialy alterd output:      |      exact same as input? {row['expected output']==row['predicted']}")
    print(row['predicted'], '\n')

ℹ Artificialy alterd output:      |      exact same as input? True
famotidine @CHEMICAL@ delirium @DISEASE@ @CID@ 

ℹ Artificialy alterd output:      |      exact same as input? True
indomethacin ; idm @CHEMICAL@ hypotension @DISEASE@ @CID@ 

ℹ Artificialy alterd output:      |      exact same as input? True
tacrolimus @CHEMICAL@ systemic sclerosis ; ssc @DISEASE@ @CID@ corticosteroid ; corticosteroids @CHEMICAL@ systemic sclerosis ; ssc @DISEASE@ @CID@ cyclosporine @CHEMICAL@ thrombotic microangiopathy @DISEASE@ @CID@ 

ℹ Artificialy alterd output:      |      exact same as input? True
methamphetamine @CHEMICAL@ psychosis ; psychotic symptoms @DISEASE@ @CID@ methamphetamine @CHEMICAL@ bipolar disorder @DISEASE@ @CID@ methamphetamine @CHEMICAL@ antisocial personality disorder ; antisocial personality @DISEASE@ @CID@ 

ℹ Artificialy alterd output:      |      exact same as input? True
levodopa @CHEMICAL@ dyskinetic ; dyskinesias ; lids ; abnormal involuntary movements @DISEASE@ @CID@ 



### Extracting triples from actual model output

In [17]:
test_eval_dataset = test_eval_dataset_trained_gc1

re_labels = ['@CID@']
for i in test_eval_dataset:
    text = i['predicted']
    try:
        extract_relation_triples(text, re_labels)
    except ValueError:
        msg.info("expected output:")
        print(i["expected output"])
        msg.info("predicted output:")
        print(text)
        print('\n')

ℹ expected output:
phosphorus @CHEMICAL@ cholestatic ; cholestasis @DISEASE@ @CID@ phosphorus @CHEMICAL@ acute hepatitis @DISEASE@ @CID@ phosphorus @CHEMICAL@ acute liver failure @DISEASE@ @CID@
ℹ predicted output:
yellow phosphorus ; yellow phosphorus @CHEMICAL@ cholestasis @DISEASE@ @CID@ yellow phosphorus ; yellow phosphorus @CHEMICAL@ hepatotoxicity ; hepatitis @DISEASE@ @CID@ yellow phosphorus ; yellow phosphorus @CHEMICAL@ fireworks @CHEMICAL@ cholestasis @DISEASE@ @CID@ yellow phosphorus ; yellow phosphorus @CHEMICAL@ cholestasis @DISEASE@ @CID@


⚠  len entity text 5 | ['lomustine ; ccnu @CHEMICAL@ neutropenia
@DISEASE@ ', ' lomustine ; ccnu @CHEMICAL@ hemorrhagic @DISEASE@ ', ' lomustine
; ccnu @CHEMICAL@ cystitis @DISEASE@ ', ' cyclophosphamide ; ctx @CHEMICAL@
neutropenia @DISEASE@ ', ' cyclophosphamide ; cnu @CHEMICAL@ hemorhagicnu
@CHEMICAL@ cystitis @']
⚠  len relation labels 4 | ['@CID@', '@CID@', '@CID@', '@CID@']
ℹ expected output:
lomustine ; ccnu @CHEMICAL@ neutropen

### Changing `extract_relation_triples()` to deal with unstructured text

In [18]:
def extract_relation_triples(text: str, ner_labels: list[str], re_labels: list[str], keep_coreforents: bool = False) -> list[dict]:
    '''
    This function extracts the relationship triples out of structerd text. 
    This function assumes that the NER labels are in this structure: @label@
    
    input:
    text: The structerd text as a string.
    re_labels: The relationship labels.

    returns:
    A list of dictionaries
    '''
    ##### Check if text is structered #####
    split_on_space_text = text.split(" ")
    
    # check if text ends with a relation label
    if split_on_space_text[-1] not in re_labels:
        raise ValueError(f"Text is unstructured: '{text}'\nText should end with a relationship label found in re_labels: {re_labels}.\n")
        
    # Check if text has atleast two entity labels and one relation label
    count_ner_labels = sum([split_on_space_text.count(label) for label in ner_labels])
    count_re_labels = sum([split_on_space_text.count(label) for label in re_labels])
    if count_ner_labels < 2 or count_re_labels < 1:
        raise ValueError(f"Text is unstructured: '{text}'\nText should have atleast 2 ner_labels: {ner_labels} and 1 re_label: {re_labels} to make a relationship.\n")
        
    # Check if text has the right amount of entity and relation labels
    if count_re_labels*2 != count_ner_labels:
        raise ValueError(f"Text is unstructured: '{text}'\nText should have 2 times the ner_labels: {ner_labels} then there are re_label: {re_labels}. currently: ner labels: {count_ner_labels} | re labels: {count_re_labels}\n")
    
    ##### Extracting relation triples #####
    # Split the input text into relation segments
    relation_segments = split_on_labels(text, re_labels)
    
    # Remove the last empty segment if it exists
    if not relation_segments[-1].strip():
        relation_segments = relation_segments[:-1]

    # Map relation label to entity text
    entity_texts = relation_segments[::2] # All uneven elements
    relation_labels = relation_segments[1::2] # All even elements
        
    # Initialize a list to hold the relation triples
    relations = []
    
    for entity_text, re_label in zip(entity_texts, relation_labels):
        # Split head and tail entities and their labels
        head_ent, tail_ent = [handle_coreforents(ent, keep_coreforents) for ent in re.findall(r'(.+?)\s@(\w+)@', entity_text)]
        # print(f"head_ent {head_ent} | tail_ent {tail_ent}") #DEBUG
        
        re_label = re_label.split('@')[1]
        relations.append({
            're_label':re_label,
            'head_ent': {'label':head_ent[1], 'text':head_ent[0]},
            'tail_ent': {'label':tail_ent[1], 'text':tail_ent[0]}
        })
    
    return relations

In [19]:
test_eval_dataset = test_eval_dataset_trained
re_labels = ['@CID@']
ner_labels = ['@CHEMICAL@', '@DISEASE@']
for i in test_eval_dataset:
    text = i['predicted']
    try:
        extract_relation_triples(text, ner_labels, re_labels)
    except ValueError as error:
        print(error)

Text is unstructured: 'yellow phosphorus ; yellow phosphorus @CHEMICAL@ cholestasis @DISEASE@ @CID@ yellow phosphorus ; yellow phosphorus @CHEMICAL@ hepatotoxicity ; hepatitis @DISEASE@ @CID@ yellow phosphorus ; yellow phosphorus @CHEMICAL@ fireworks @CHEMICAL@ cholestasis @DISEASE@ @CID@ yellow phosphorus ; yellow phosphorus @CHEMICAL@ cholestasis @DISEASE@ @CID@'
Text should have 2 times the ner_labels: ['@CHEMICAL@', '@DISEASE@'] then there are re_label: ['@CID@']. currently: ner labels: 9 | re labels: 4

Text is unstructured: 'lomustine ; ccnu @CHEMICAL@ neutropenia @DISEASE@ @CID@ lomustine ; ccnu @CHEMICAL@ hemorrhagic @DISEASE@ @CID@ lomustine ; ccnu @CHEMICAL@ cystitis @DISEASE@ @CID@ cyclophosphamide ; ctx @CHEMICAL@ neutropenia @DISEASE@ @CID@ cyclophosphamide ; cnu @CHEMICAL@ hemorhagicnu @CHEMICAL@ cystitis @'
Text should end with a relationship label found in re_labels: ['@CID@'].

Text is unstructured: 'fluocinolone acetonide intravitreal implant ; 0.59 mg @CHEMICAL@ ocul

In [20]:
clf_metrics = evaluate.combine(["f1", "precision", "recall"])

In [21]:
clf_metrics.compute(
    predictions=[0,1,1],
    references=[1,0,1]
)

{'f1': 0.5, 'precision': 0.5, 'recall': 0.5}

All of these metrics take input as a list of ints.

documentation:
- [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy)
- [precision](https://huggingface.co/spaces/evaluate-metric/precision)
- [recall](https://huggingface.co/spaces/evaluate-metric/recall)
- [f1](https://huggingface.co/spaces/evaluate-metric/f1)

so the extract relation triples needs to work on the input ids. Or the text needs to be converted to ints

Because we're doing a relation extraction from a document, evaluating the output will be a bit different from usual methods. This method will be based on if the set of entities or relation in the predictions are also found in the set of entities and relations in the reference. The sequence of entities and relations are irrelivant. 

### Named entity recoginition scoring

In [22]:
# starting point
index = 4
relation_triples[index]

[{'re_label': 'CID',
  'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')},
  'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}},
 {'re_label': 'CID',
  'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')},
  'tail_ent': {'label': 'DISEASE', 'text': ('proteinuria',)}},
 {'re_label': 'CID',
  'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')},
  'tail_ent': {'label': 'DISEASE', 'text': ('hypertension',)}}]

In [23]:
# predictions = relation_triples[index]
# references = relation_triples[index]

# ### Define group of entities in the reference
# entities_text = list()
# for relation in references:
#     nested_list = list(relation.values())[0] # Elements can be list if there is a coreferent mention
#     # This next list comprehension is barely readable but if an element is a sublist it is flattend 
#     # if an element is a string it's kept as it is. 
#     entities_text.extend([item for sublist in nested_list.values() for item in (sublist if isinstance(sublist, list) else [sublist])])

# ### Check if a prediction is in the group of reference entities
# for pred in predictions:
#     for pred_ent in list(pred.values())[0].values():
#         if isinstance(pred_ent, list):
#         else:
#             if pred_ent in entities_text:
#                 pass
#     # Remove entity texts from the group if there is a match
# # Check if the labels match
# # 

### Relation extraxtion scoring

In [24]:
# Load tokenizer to access input ids
model_name = config['model_name']
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, legacy=False)

In [25]:
# voorbeeld
msg.info(f"Output in text:")
print(relations_in_structerd_text[0]+'\n')
input_ids=tokenizer(relations_in_structerd_text[0], return_tensors="pt").input_ids
msg.info(f"Output as input ids:")
print(input_ids)
print() 

msg.info("Input ids of entity labels")
for key, value in entity_types.items():
    input_ids = tokenizer(value, return_tensors="pt").input_ids
    tokens = tokenizer.tokenize(value)
    print(f"{key} : {value}\ntokens:{tokens}\ninput ids: {input_ids[0][:-1]}"+"\n")

msg.info("Input ids of relation labels")
for key, value in rel_types.items():
    input_ids = tokenizer(value, return_tensors="pt").input_ids
    tokens = tokenizer.tokenize(value)
    print(f"{key} : {value}\ntokens:{tokens}\ninput ids: {input_ids[0][:-1]}"+"\n")

ℹ Output in text:
alpha-methyldopa @CHEMICAL@ hypotensive @DISEASE@ @CID@

ℹ Output as input ids:
tensor([[  491,  6977,    18, 22758,    26,    32,   102,     9,  3320, 13717,
           329, 23936,  1741, 10950,   324,     7,   757,  3320,   308, 19056,
         17892,  1741,  3320,   254,  4309,  1741,     1]])

ℹ Input ids of entity labels
chemical : @CHEMICAL@
tokens:['▁@', 'CHE', 'M', 'ICAL', '@']
input ids: tensor([ 3320, 13717,   329, 23936,  1741])

disease : @DISEASE@
tokens:['▁@', 'D', 'ISE', 'ASE', '@']
input ids: tensor([ 3320,   308, 19056, 17892,  1741])

ℹ Input ids of relation labels
chemical induced disease : @CID@
tokens:['▁@', 'C', 'ID', '@']
input ids: tensor([3320,  254, 4309, 1741])



We can use the unique sequence of input ids of the entity and relation labels to extract the input ids of the entity text.

## Custom implementation of precision, recall and f1 for RE:

In [26]:
def match_re_relaxed(predicted_triple, references):

    pred_head_mentions = predicted_triple["head_ent"]["text"]
    pred_tail_mentions = predicted_triple["tail_ent"]["text"]

    for head_mention in pred_head_mentions:
        for reference_triple in references:
            if head_mention in reference_triple["head_ent"]["text"] and reference_triple["head_ent"]["label"]==predicted_triple["head_ent"]["label"]: # Head entity match
                for tail_mention in pred_tail_mentions:
                    if tail_mention in reference_triple["tail_ent"]["text"] and reference_triple["tail_ent"]["label"]==predicted_triple["tail_ent"]["label"]: # Tail entity match
                        return (True, reference_triple)
    return (False, None)

In [27]:
# Test match_re_relaxed()
tri_1 = relation_triples[2][0]
tri_2 = relation_triples[4][0]
refs = relation_triples[4]

print(f"Triple 1: {tri_1}")
print(f"Triple 2: {tri_2}\n")
print(f"references: {refs}\n\n")

print(match_re_relaxed(tri_1, refs)) # should be false
print(match_re_relaxed(tri_2, refs)) # should be True

Triple 1: {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('suxamethonium', 'suxamethonium chloride', 'sch')}, 'tail_ent': {'label': 'DISEASE', 'text': ('fasciculations', 'fasciculation')}}
Triple 2: {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}}

references: [{'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}}, {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('proteinuria',)}}, {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('hypertension',)}}]


(False, None)
(True, {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}})


In [28]:
def match_re_strict(predicted_triple, references):

    pred_head_mentions = predicted_triple["head_ent"]["text"]
    pred_tail_mentions = predicted_triple["tail_ent"]["text"]

    for reference_triple in references:
        if set(pred_head_mentions) == set(reference_triple["head_ent"]["text"]) and reference_triple["head_ent"]["label"]==predicted_triple["head_ent"]["label"]: # Head entity match
            for tail_mention in pred_tail_mentions:
                    if set(pred_tail_mentions) == set(reference_triple["tail_ent"]["text"]) and reference_triple["tail_ent"]["label"]==predicted_triple["tail_ent"]["label"]: # Tail entity match
                        return (True, reference_triple)
    return (False, None)

In [29]:
# Test match_re_strict()
tri_1 = relation_triples[2][0]
tri_2 = relation_triples[4][0]
refs = relation_triples[4]

print(f"Triple 1: {tri_1}")
print(f"Triple 2: {tri_2}\n")
print(f"references: {refs}\n\n")

print(match_re_strict(tri_1, refs)) # should be false
print(match_re_strict(tri_2, refs)) # should be True

Triple 1: {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('suxamethonium', 'suxamethonium chloride', 'sch')}, 'tail_ent': {'label': 'DISEASE', 'text': ('fasciculations', 'fasciculation')}}
Triple 2: {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}}

references: [{'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}}, {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('proteinuria',)}}, {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('hypertension',)}}]


(False, None)
(True, {'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('lithium', 'li')}, 'tail_ent': {'label': 'DISEASE', 'text': ('chronic renal failure',)}})


In [30]:
def re_metric(predictions: list[str], references: list[str], ner_labels: list[str], re_labels: list[str], coferent_matching: str ="relaxed"):
    
    tp = 0 # True positive count
    fp = 0 # False positive count
    fn = 0 # False negative count
    
    unstructured_text_count = 0

    if coferent_matching == "no":
        keep_coreforents = False
    else:
        keep_coreforents = True
    
    # Transform structed text into relationship triples
    for pred_text, ref_text in zip(predictions, references):
        try:
            predicted_triples = extract_relation_triples(pred_text, ner_labels, re_labels, keep_coreforents=keep_coreforents)
        except ValueError: # Text is unstructured
            unstructured_text_count += 1
            continue
    
        references = extract_relation_triples(ref_text, ner_labels, re_labels, keep_coreforents=keep_coreforents)

        # Determine matches between predicted and reference triples
        for predicted_triple in predicted_triples:
            # print(f"Checking if {predicted_triple}\nin") # DEBUG
            # print(f"references{references}\n") # DEBUG
            if coferent_matching == "relaxed":
                is_match, matched_ref_triple = match_re_relaxed(predicted_triple, references)
                if is_match: 
                    tp = tp + 1
                    # print(f"True!") # DEBUG
                    references.remove(matched_ref_triple)
                else:
                    fp = fp + 1
                    # print(f"False!") # DEBUG
                    
            elif coferent_matching == "strict" or "no":
                is_match, matched_ref_triple = match_re_strict(predicted_triple, references)
                if is_match: 
                    tp = tp + 1
                    # print(f"True!") # DEBUG
                    references.remove(matched_ref_triple)
                else:
                    fp = fp + 1
                    # print(f"False!") # DEBUG
        
                
        # False negative
        # print(f"Counting false negatives: {len(references)} from: {references} \n") #DEBUG
        fn+=len(references)
        # print(f"Current counts: tp:{tp} | fp:{fp} | fn:{fn} \n\n") # DEBUG

    # Calculate metrics
    if (tp+fp) == 0: precision=0.0
    else: precision = tp/(tp+fp)

    if (tp+fn) == 0: recall=0.0
    else: recall = tp/(tp+fn)

    if (precision+recall) == 0: f1=0.0
    else: f1 = 2 * ((precision*recall)/(precision+recall))

    unstructured_text = unstructured_text_count/len(predictions)

    return {'re_precision':precision, 're_recall':recall, 're_f1':f1, 'unstructured':unstructured_text}

In [31]:
# Testing with dataset with know scores
test_eval_dataset = test_eval_dataset_re53_ner53
re_labels = ['@CID@']
ner_labels = ['@CHEMICAL@', '@DISEASE@']
predictions = [i['predicted'] for i in test_eval_dataset]
references = [i['expected output'] for i in test_eval_dataset]

result = re_metric(predictions, references, ner_labels, re_labels, coferent_matching="relaxed")
result = {k: round(v * 100, 4) for k, v in result.items()}
print(f"Expected score should be 52.9412% for re and ner: {result}")

Expected score should be 52.9412% for re and ner: {'re_precision': 52.9412, 're_recall': 52.9412, 're_f1': 52.9412, 'unstructured': 0.0}


In [43]:
# Testing
test_eval_dataset = test_eval_dataset_trained_3b
re_labels = ['@CID@']
ner_labels = ['@CHEMICAL@', '@DISEASE@']
predictions = [i['predicted'] for i in test_eval_dataset]
references = [i['expected output'] for i in test_eval_dataset]

result = re_metric(references, references, ner_labels, re_labels)
result = {k: round(v * 100, 4) for k, v in result.items()}
print(f"Result with references as input: {result}")

result = re_metric(predictions, references, ner_labels, re_labels)
result = {k: round(v * 100, 4) for k, v in result.items()}
print(f"Actual results: {result}")

Result with references as input: {'re_precision': 100.0, 're_recall': 100.0, 're_f1': 100.0, 'unstructured': 0.0}
Actual results: {'re_precision': 74.7126, 're_recall': 64.8056, 're_f1': 69.4074, 'unstructured': 2.6}


## Custom implementation of precision, recall and f1 for NER:
This function should take a list of predicted outputs and a list of references. The lists are a list of strings, the strings being the decoded output of the model (using `tokenizer.batch_decode()`)

The output should be a dictionary with `key : value` pairs of `name metric : value of metric`

In [33]:
def get_group(relation_triples):
    group = []
    for rel in relation_triples:
        group.append(rel['head_ent'])
        group.append(rel['tail_ent'])
    return group

In [34]:
# testing get_group
print(f"Relation triples: {relation_triples[2]}")
get_group(relation_triples[2])

Relation triples: [{'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('suxamethonium', 'suxamethonium chloride', 'sch')}, 'tail_ent': {'label': 'DISEASE', 'text': ('fasciculations', 'fasciculation')}}]


[{'label': 'CHEMICAL',
  'text': ('suxamethonium', 'suxamethonium chloride', 'sch')},
 {'label': 'DISEASE', 'text': ('fasciculations', 'fasciculation')}]

In [35]:
def split_coferents(ent):
    '''
    This function splits a entitiy with a coferent mention into two entities for each entity form.
    '''
    if isinstance(ent['text'], tuple): # Check if the entity has coferent mentions
        return tuple([{"label":ent["label"], "text":ent['text'][i]} for i in range(len(ent['text']))])
    else:
        return (ent,)

In [36]:
# Testing split_coferents
print(f"A single ent: {relation_triples[1][0]['head_ent']}")
split_coferents(relation_triples[1][0]['head_ent'])

A single ent: {'label': 'CHEMICAL', 'text': ('lidocaine',)}


({'label': 'CHEMICAL', 'text': 'lidocaine'},)

In [37]:
def map_coferents(group):
    '''
    This function maps all forms of a coferent mentions to all it's other forms for all coferent mentions in a group of relationships.
    '''
    result = {}
    group = [split_coferents(ent) for ent in group] # Split coferents into two entities
    for ent in group:
        if isinstance(ent, tuple):
            for i in range(len(ent)):
                result[frozenset(ent[i].items())] = ent
        else:
            result[frozenset(ent.items())] = (ent,)

    return result

In [38]:
# testing map_coferents
print(f"Relation triples: {relation_triples[3]}")
group = get_group(relation_triples[3])
map_coferents(group)

Relation triples: [{'re_label': 'CID', 'head_ent': {'label': 'CHEMICAL', 'text': ('scopolamine', 'hyoscine')}, 'tail_ent': {'label': 'DISEASE', 'text': ('overdosage',)}}]


{frozenset({('label', 'CHEMICAL'),
            ('text', 'scopolamine')}): ({'label': 'CHEMICAL',
   'text': 'scopolamine'}, {'label': 'CHEMICAL', 'text': 'hyoscine'}),
 frozenset({('label', 'CHEMICAL'),
            ('text', 'hyoscine')}): ({'label': 'CHEMICAL',
   'text': 'scopolamine'}, {'label': 'CHEMICAL', 'text': 'hyoscine'}),
 frozenset({('label', 'DISEASE'),
            ('text', 'overdosage')}): ({'label': 'DISEASE',
   'text': 'overdosage'},)}

In [39]:
def ner_metric(predictions: list[str], references: list[str], ner_labels: list[str], re_labels: list[str], coferent_matching: str ="relaxed") -> dict:
    '''
    Calculates the precision, recall and f1-score for document named entity recognition. 
    input:
        predictions: 
            List of decoded outputs of the model
        references: 
            List of decoded gold data
        coferent_matching: 
            Wheter to use the coferent mentions to match named entities. can be either "relaxed", "strict" or "no".
            "relaxed" Meaning that all coferent mentions might be used to match a predicted named entity to a reference entity
            "strict"  Meaning the model needs to have all coferent mentions correct to count as a match. (including the sequence)
            "no"      Meaning that coferent mentions are ignored, and only the first mentions are used.
        re_labels
            Which relation extraxtion labels are used.
    output
        a dictionary with the key value pairs of metric_name : metric value
    '''
    if coferent_matching not in ["relaxed", "strict", "no"]: 
        raise ValueError(f"'{coferent_matching}' is not a valid value for coferent_matching, Please choose one of {["relaxed", "strict", "no"]}.")

    if coferent_matching == "no":
        keep_coferents = False
    else:
        keep_coferents = True
    
    tp = 0 # True positive count
    fp = 0 # False positive count
    fn = 0 # False negative count
    unstructured_text_count = 0
    
    for pred_text, ref_text in zip(predictions, references):
        # Define groups
        try:
            pred_group = get_group(extract_relation_triples(pred_text, ner_labels, re_labels, keep_coferents))
        except ValueError:
            # Should be a logging statement here
            continue # Skip this row entirely
        
        ref_group = get_group(extract_relation_triples(ref_text, ner_labels, re_labels, keep_coferents))

        if coferent_matching == "relaxed":
            # Create mapping from a coferent mentions to all coferent mentions
            mapping_coferent = map_coferents(ref_group)
            
            # Split entities in the reference group
            ref_group = [split_coferents(ent) for ent in ref_group] # Split coferents into multiple entities 
            ref_group = [item for sublist in ref_group for item in (sublist if isinstance(sublist, tuple) else [sublist])] # Flatten list
        
            # print(f"pred_group: {pred_group}\n\nref_group: {ref_group}\n") #DEBUG
            # print(f"mapping: {mapping_coferent}\n\n\n") # DEBUG
        
        checked_coferent_pred = []
        for ent in pred_group:
            # print(f"entity: {ent}") # DEBUG
            # print(f"ref_group: {ref_group} ") # DEBUG

            if coferent_matching == "relaxed":
                # Split coferent entity
                ent_forms = split_coferents(ent)
            else:
                ent_forms = [ent]

            # print(f"\nStarting entity checking, ent_forms: {ent_forms}\n") # DEBUG
            for ent_form in ent_forms:
                # print(f"Checking if {ent_form} in {ref_group}") # DEBUG
                if ent_form in ref_group: # True positive
                    tp=tp+1
                    # print(f"True! \n") # DEBUG
                    
                    # Remove all instances of the coferent mentions
                    if coferent_matching == "relaxed": 
                        [ref_group.remove(i) for i in mapping_coferent[frozenset(ent_form.items())]] # Remove all coferent mentions from the reference group
                        # print(f"Removing coferent mentions from reference group: {[i for i in mapping_coferent[frozenset(ent_form.items())]]}\n") #DEBUG
                        checked_coferent_pred.extend([i for i in mapping_coferent[frozenset(ent_form.items())]]) # Remember which coferent mentions have been checked
                    else: 
                        ref_group.remove(ent) 
                    break # A match was found so we move on to the next entity
                    
                elif ent_form not in ref_group and ent_form not in checked_coferent_pred: # False positive
                    fp=fp+1
                    # print(f"False! \n") #DEBUG
                    break # A mismatch was found so we move on to the next entity

        # print(f"Counting false negatives. Current ref group: length:{len(ref_group)} | {ref_group}\n") # DEBUG
        # [ref_group.remove(i) for i in checked_coferent_pred if i in ref_group]
        # print(f"ref group after removeal of checked coferents: length:{len(ref_group)} | {ref_group}\n") # DEBUG

        # if coferent_matching == "relaxed": # WORK NEEDED. RELAXED MATCHING BASED ON COFERENT MENTIONS DOES NOT WORK YET!!!
            # print(f"checked_coferent_pred: {checked_coferent_pred}") #DEBUG
            # Remove all checked entities before counting false negatives
            
        fn=fn+len(ref_group) # False negative 
        # print(f"TP: {tp}, FP: {fp}, FN: {fn} \n\n\n") #DEBUG
    
    # Calculate metrics
    if (tp+fp) == 0: precision=0.0
    else: precision = tp/(tp+fp)
    
    if (tp+fn) == 0: recall=0.0
    else: recall = tp/(tp+fn)
    
    if (precision+recall) == 0: f1=0.0
    else: f1 = 2 * ((precision*recall)/(precision+recall))
    
    return {'ner_precision':precision, 'ner_recall':recall, 'ner_f1':f1}

In [40]:
# Testing with dataset with know scores
test_eval_dataset = test_eval_dataset_re53_ner53
re_labels = ['@CID@']
ner_labels = ['@CHEMICAL@', '@DISEASE@']
predictions = [i['predicted'] for i in test_eval_dataset]
references = [i['expected output'] for i in test_eval_dataset]

result = ner_metric(predictions, references, ner_labels, re_labels, "strict")
result = {k: round(v * 100, 4) for k, v in result.items()}
print(f"Expected score should be 52.9412% for re and ner: {result}")

Expected score should be 52.9412% for re and ner: {'ner_precision': 52.9412, 'ner_recall': 52.9412, 'ner_f1': 52.9412}


In [41]:
test_eval_dataset = test_eval_dataset_trained_3b
re_labels = ['@CID@']
ner_labels = ['@CHEMICAL@', '@DISEASE@']
predictions = [i['predicted'] for i in test_eval_dataset]
references = [i['expected output'] for i in test_eval_dataset]

result = ner_metric(references, references, ner_labels, re_labels)
result = {k: round(v * 100, 4) for k, v in result.items()}
print(f"Result with references as input: {result}")

result = ner_metric(predictions, references, ner_labels, re_labels, "strict")
result = {k: round(v * 100, 4) for k, v in result.items()}
print(f"Actual results: {result}")

Result with references as input: {'ner_precision': 100.0, 'ner_recall': 100.0, 'ner_f1': 100.0}
Actual results: {'ner_precision': 68.6207, 'ner_recall': 59.5214, 'ner_f1': 63.748}


## Loading the model to evaluate 

In [42]:
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    device_map=device_map

SyntaxError: incomplete input (1774832362.py, line 3)